In [1]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


Note: you may need to restart the kernel to use updated packages.


In [2]:
# First authenticate your identity using the JSON credentials and provide access to your Google Drive.
# Based off of https://developers.google.com/drive/api/quickstart/python

from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']


def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
# Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('drive', 'v3', credentials=creds)

# Call the Drive v3 API - calling 10 files + id/token
        results = service.files().list(
            pageSize=10, fields="nextPageToken, files(id, name)").execute()
        items = results.get('files', [])

        if not items:
            print('No files found.')
            return
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))
    except HttpError as error:
# TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')


if __name__ == '__main__':
    main()

Files:
college basketball (1t610Dd-bL8TWBkztVeIuT90QKXc3dkMm)
data (1xhghbG5r6eR6_4MrkjyQGq4fPTBGySIo)
documentation (1TMTZ3ySX_ggKnzuLjRyQrhhXM2YzLih5fdEcN52QFKc)
Timeline | To do list (1inKRZ3jvG8fi4nEsl1vZg07lqcY_utqf5Yia5A1dB68)
resources (1PYnOGFa0pyTqv58fkRd7BGwRO8rTij221W0toqonb60)
brainstorming (1pDzYFMRMvZSqZQ3SdJgpTsRecLP3V20EUDLnXgPnRg0)
Darrenshootinglessthan70project (1NufGSFtVnSn2njQrWj_n5xMwIcEZQH_e)
2023 KCul Grant Lists.docx (1NB1w3wjiXcNVQA6-61pV0gfIPdhR5Vun)
KCul Grant Paul  (1iShTLC6LnMh0AXiR7YX4ugbOC4c92ezW)
Submission Guidelines and Tips (1aVGmC4WE-4kPFJDAdSTedVQ1Mkhecaqyh_BpA4OpqFk)


In [3]:
# Second download the file indicated by the file ID to your Jupyter Notebook folder.
# based off of https://developers.google.com/drive/api/v3/manage-downloads

from __future__ import print_function

import io

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload


def download_file(real_file_id):
    """Downloads a file
    Args:
        real_file_id: ID of the file to download
    Returns : IO object with location.

    Load pre-authorized user credentials from the environment.
    TODO(developer) - See https://developers.google.com/identity
    for guides on implementing OAuth2 for the application.
    """
    creds, _ = google.auth.default()

    try:
        # create drive api client
        service = build('drive', 'v3', credentials=creds)

        file_id = real_file_id

        # pylint: disable=maybe-no-member
        request = service.files().get_media(fileId=file_id)
        file = io.BytesIO()
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(F'Download {int(status.progress() * 100)}.')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None

    return file.getvalue()


if __name__ == '__main__':
    download_file(real_file_id='1KuPmvGq8yoYgbfW74OENMCB5H0n_2Jm9')

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.